In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import nltk
import pandas as pd
import os
from eunjeon import Mecab
from ast import literal_eval

xname = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#8_clean_text'}.xlsx")
df = pd.read_excel(xname, index_col=0)
mecab = Mecab()
corpus = []
TEXT = []
count = 0

for i in range(100):
    text = literal_eval(df.loc[i, 'text'])
    corpus.append( ". ".join([" ".join([f for f in mecab.nouns(e) if len(f)>1]) for e in text]))
     
vectorizer = TfidfVectorizer()
sp_matrix = vectorizer.fit_transform(corpus)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx

for i, sent in enumerate(corpus):
    #print('====== document[%d] ======' % i)
    result =  set([ (token, sp_matrix[i, word2id[token]]) for token in sent.split() if sp_matrix[i, word2id[token]] > 0.02 ])
    print(result)
    count += len(result)
    
count/100

{('위헌법률심판', 0.039343712875012905), ('예측', 0.032019357986871805), ('사유', 0.024381492973496708), ('수절', 0.039343712875012905), ('정한', 0.030561973622647218), ('고려', 0.030014745730257995), ('법률', 0.036102634205071094), ('정부', 0.029329772699918558), ('가중', 0.09619336693194057), ('죄형', 0.1180311386250387), ('형벌', 0.10140915687776708), ('구성요건', 0.03380305229258903), ('책임', 0.05004262608044668), ('구체', 0.06816519338322383), ('동시', 0.020548033447552874), ('구법', 0.07220526841014219), ('확대', 0.030561973622647218), ('법령', 0.061123947245294435), ('혼선', 0.039343712875012905), ('직권', 0.032019357986871805), ('처단형', 0.048762985946993416), ('실무', 0.18051317102535547), ('절도', 0.19124626466893788), ('유죄', 0.02571683705046228), ('지방', 0.09515644846997873), ('결론', 0.04544346225548255), ('형법', 0.07868742575002581), ('대법원', 0.06403871597374361), ('개정', 0.21856715962164328), ('반복', 0.022721731127741274), ('원칙', 0.17597863619951135), ('재량', 0.036102634205071094), ('재판소', 0.036102634205071094), ('과정', 0.02438149

{('용도', 0.03863133042368357), ('침상', 0.020779421695263235), ('트럭', 0.020779421695263235), ('전동', 0.04155884339052647), ('기능', 0.03228267447772187), ('승인', 0.05970724291608088), ('국토', 0.03570623242428345), ('출입구', 0.020779421695263235), ('연결', 0.04328867479674045), ('시행', 0.0288591165311603), ('마찬가지', 0.03382211252580396), ('튜닝', 0.17160878449784203), ('본체', 0.020779421695263235), ('교통안전공단', 0.020779421695263235), ('변경', 0.05751626391229322), ('상하차', 0.04155884339052647), ('관리법', 0.08455528131450989), ('오인', 0.030981098153798847), ('물품', 0.046471647230698264), ('불편', 0.020779421695263235), ('취침', 0.04155884339052647), ('교통부', 0.020779421695263235), ('성능', 0.020779421695263235), ('위법', 0.02796950155956094), ('인식', 0.02454594361299936), ('화물', 0.11440585633189468), ('후방', 0.020779421695263235), ('자동차', 0.40966598442792235), ('설비', 0.03570623242428345), ('재량', 0.03813528544396489), ('부착', 0.1249718134849921), ('공간', 0.046471647230698264), ('승차', 0.05720292816594734), ('정원', 0.038135285443

{('근로자', 0.07772664306257723), ('지배', 0.08549930736883495), ('최종', 0.1644948954113806), ('소급', 0.033881790564336246), ('제안', 0.031572554523360476), ('개별', 0.08208864176073723), ('체결', 0.05882012481628885), ('항소', 0.022802771124423124), ('노조원', 0.03109065722503089), ('노조', 0.14475550796201492), ('영향', 0.03913473737341252), ('이익', 0.0266702866953386), ('결의', 0.029110316605126252), ('실제', 0.023528049926515544), ('위원장', 0.02183273745384469), ('합의', 0.03143019140053678), ('후생', 0.033881790564336246), ('대우', 0.025411342923252183), ('개정', 0.0411740873714022), ('오해', 0.054408650143804055), ('노동조합법', 0.042352238205420306), ('기본', 0.020970294184251132), ('조합원', 0.12501612051264924), ('임금', 0.08733094981537876), ('개입', 0.09327197167509267), ('동의', 0.02525804361868838), ('탈퇴', 0.1101158193340928), ('수용', 0.18193947878203906), ('노동', 0.1378712254181656), ('시기', 0.029110316605126252), ('오인', 0.037887065428032565), ('노사', 0.042352238205420306), ('예상', 0.024338842645916254), ('소속', 0.02298048626567313

{('사용', 0.03399171402464312), ('사회', 0.04136127698940622), ('관례', 0.03832651541415145), ('활용', 0.041767240844224195), ('도장', 0.03832651541415145), ('주말', 0.041767240844224195), ('가치', 0.032444556441002), ('의문', 0.035885281871074745), ('표현', 0.2400265831834024), ('시정', 0.029003831010929258), ('지하', 0.032444556441002), ('일방', 0.035885281871074745), ('황금', 0.041767240844224195), ('입주', 0.12530172253267258), ('부과', 0.029003831010929258), ('앞전', 0.041767240844224195), ('제안서', 0.041767240844224195), ('감정', 0.064889112882004), ('오늘', 0.041767240844224195), ('변호인', 0.04136127698940622), ('반복', 0.024121363924775853), ('동의', 0.03113645334276165), ('밴드', 0.19163257707075726), ('불합리성', 0.041767240844224195), ('적시', 0.035885281871074745), ('도구', 0.03399171402464312), ('인격', 0.041767240844224195), ('역서', 0.035885281871074745), ('추상', 0.035885281871074745), ('누구', 0.024669029621420927), ('게시판', 0.035885281871074745), ('주민', 0.028109755051493674), ('실체', 0.032444556441002), ('사항', 0.047213822161732553

{('변제', 0.15186519958349692), ('동기', 0.04695999775878594), ('합의', 0.06743479227676023), ('기망', 0.17127262979495234), ('여행사', 0.400237844879705), ('항공', 0.4361688058886632), ('번영', 0.06245740372245672), ('성행', 0.022961719657713195), ('사기', 0.07198823140597876), ('왕복', 0.06670630747995084), ('초과', 0.03944447675792755), ('계약자', 0.06245740372245672), ('권고', 0.06169964018843163), ('사무실', 0.16434886941281188), ('대상', 0.03944447675792755), ('반성', 0.03000562220149635), ('여행', 0.3335315373997542), ('영위', 0.06245740372245672), ('경합범', 0.044922025295622794), ('별지', 0.07198823140597876), ('보라카이', 0.07269480098144387), ('보라', 0.06670630747995084), ('편취', 0.034811929553899064), ('결정', 0.026385300211647256), ('고소', 0.05048041671947065), ('가이드', 0.07269480098144387), ('거래', 0.08217443470640594), ('하한', 0.05916169792626588), ('피고인', 0.03360886138635725), ('의뢰', 0.05646891022096368), ('제반', 0.024861326592722784), ('불리', 0.03129073170965989), ('호텔', 0.13341261495990167), ('직전', 0.06245740372245672), ('잠적

86.76